# 🎵 Song Release Year Prediction - Model Training

This notebook trains multiple regression models to predict the year a song was released based on its audio features.

**Dataset Context:** Audio features of songs (timbre, pitch, rhythm, etc.) used to predict release year.

## Models to Train:
1. Linear Regression
2. Ridge Regression
3. Lasso Regression
4. ElasticNet Regression
5. Decision Tree Regressor
6. Random Forest Regressor
7. Gradient Boosting Regressor
8. XGBoost Regressor (if available)
9. Support Vector Regressor (SVR)
10. K-Nearest Neighbors Regressor

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from datetime import datetime

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Try to import XGBoost
try:
    from xgboost import XGBRegressor
    xgboost_available = True
    print("✓ XGBoost is available")
except ImportError:
    xgboost_available = False
    print("⚠ XGBoost not available. Install with: pip install xgboost")

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
print("\n✓ All libraries imported successfully!")

✓ XGBoost is available

✓ All libraries imported successfully!


## Load Processed Data

In [2]:
# Load the scaled dataset (StandardScaler version recommended)
print("Loading processed song features data...\n")

df = pd.read_csv('cars_scaled_standard.csv')

print(f"Dataset shape: {df.shape}")
print(f"Columns (audio features): {df.shape[1]}")
print(f"Rows (songs): {df.shape[0]:,}")
print(f"\nFirst few rows:")
df.head()

Loading processed song features data...

Dataset shape: (515130, 100)
Columns (audio features): 100
Rows (songs): 515,130

First few rows:
Dataset shape: (515130, 100)
Columns (audio features): 100
Rows (songs): 515,130

First few rows:


,2001,49.94357,21.47114,73.0775,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,2.26327,feature_sum,feature_mean,feature_std,feature_min,feature_max,feature_range,num_positive,num_negative,positive_ratio
0,2001,0.897119,0.340776,1.832149,0.772068,-0.169637,-1.232206,0.800067,0.115965,1.505517,...,1.303308,-0.714015,-0.714015,-0.729524,0.377471,-0.422735,-0.437059,0.754446,-0.754444,0.754446
1,2001,1.272365,0.609508,1.401274,0.802770,0.001628,-0.724991,-0.064422,-0.072825,1.235229,...,-0.096608,-1.247273,-1.247273,-1.150659,0.713968,-1.070976,-1.063895,0.295308,-0.295307,0.295308
2,2001,0.815383,-0.066078,0.821536,0.096232,0.346899,-1.343440,0.533412,-1.127676,-0.020300,...,0.893911,-0.842589,-0.842589,-0.708272,1.065736,-0.349783,-0.496472,-0.622966,0.622968,-0.622966
3,2001,1.274568,0.816740,1.736307,0.479876,-0.422435,-0.585022,-0.724961,-1.000306,0.890946,...,1.396102,-1.263565,-1.263565,-1.229833,0.717071,-1.146780,-1.130709,-1.311672,1.311674,-1.311672
4,2001,1.203308,-0.021764,2.486225,1.388011,-0.864429,-0.765311,1.654661,-0.448923,-0.011715,...,-0.061789,-1.188286,-1.188286,-1.168220,0.749370,-1.077859,-1.076247,-0.622966,0.622968,-0.622966


## Prepare Data for Training

In [3]:
# Separate features and target
target_col = df.columns[0]  # First column is the target
X = df.drop(columns=[target_col])
y = df[target_col]

print(f"Target variable: {target_col}")
print(f"Number of features: {X.shape[1]}")
print(f"Number of samples: {X.shape[0]:,}")
print(f"\nTarget statistics:")
print(y.describe())

Target variable: 2001
Number of features: 99
Number of samples: 515,130

Target statistics:
count    515130.000000
mean       1998.396300
std          10.931639
min        1922.000000
25%        1994.000000
50%        2002.000000
75%        2006.000000
max        2011.000000
Name: 2001, dtype: float64


In [4]:
# Split data into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set size: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nFeatures: {X_train.shape[1]}")

Training set size: 412,104 samples (80.0%)
Test set size: 103,026 samples (20.0%)

Features: 99


## Define Evaluation Function

In [5]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Train and evaluate a regression model.
    
    Returns:
        dict: Dictionary containing evaluation metrics
    """
    print(f"\n{'='*80}")
    print(f"Training: {model_name}")
    print(f"{'='*80}")
    
    # Train the model
    start_time = datetime.now()
    model.fit(X_train, y_train)
    training_time = (datetime.now() - start_time).total_seconds()
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Print results
    print(f"\n📊 Training Metrics:")
    print(f"   RMSE: {train_rmse:.4f}")
    print(f"   MAE:  {train_mae:.4f}")
    print(f"   R²:   {train_r2:.4f}")
    
    print(f"\n📊 Test Metrics:")
    print(f"   RMSE: {test_rmse:.4f}")
    print(f"   MAE:  {test_mae:.4f}")
    print(f"   R²:   {test_r2:.4f}")
    
    print(f"\n⏱️  Training Time: {training_time:.2f} seconds")
    
    # Check for overfitting
    if train_r2 - test_r2 > 0.1:
        print(f"\n⚠️  Warning: Possible overfitting detected (R² difference: {train_r2 - test_r2:.4f})")
    
    return {
        'model_name': model_name,
        'model': model,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'training_time': training_time,
        'predictions': y_test_pred
    }

## Train Models

### 1. Linear Regression

In [7]:
# Linear Regression
lr_model = LinearRegression()
lr_results = evaluate_model(lr_model, X_train, X_test, y_train, y_test, "Linear Regression")


Training: Linear Regression

📊 Training Metrics:
   RMSE: 9.4608
   MAE:  6.7373
   R²:   0.2522

📊 Test Metrics:
   RMSE: 9.4113
   MAE:  6.7181
   R²:   0.2538

⏱️  Training Time: 5.65 seconds

📊 Training Metrics:
   RMSE: 9.4608
   MAE:  6.7373
   R²:   0.2522

📊 Test Metrics:
   RMSE: 9.4113
   MAE:  6.7181
   R²:   0.2538

⏱️  Training Time: 5.65 seconds


### 2. Ridge Regression

In [8]:
# Ridge Regression (L2 regularization)
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_results = evaluate_model(ridge_model, X_train, X_test, y_train, y_test, "Ridge Regression")


Training: Ridge Regression

📊 Training Metrics:
   RMSE: 9.4609
   MAE:  6.7373
   R²:   0.2522

📊 Test Metrics:
   RMSE: 9.4114
   MAE:  6.7181
   R²:   0.2538

⏱️  Training Time: 0.57 seconds

📊 Training Metrics:
   RMSE: 9.4609
   MAE:  6.7373
   R²:   0.2522

📊 Test Metrics:
   RMSE: 9.4114
   MAE:  6.7181
   R²:   0.2538

⏱️  Training Time: 0.57 seconds


### 3. Lasso Regression

In [ ]:
# Lasso Regression (L1 regularization)
lasso_model = Lasso(alpha=0.1, random_state=42, max_iter=10000)
lasso_results = evaluate_model(lasso_model, X_train, X_test, y_train, y_test, "Lasso Regression")

TypeError: Lasso.__init__() got an unexpected keyword argument 'verbose'

### 4. ElasticNet Regression

In [ ]:
# ElasticNet Regression (L1 + L2 regularization)
elasticnet_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42, max_iter=10000)
elasticnet_results = evaluate_model(elasticnet_model, X_train, X_test, y_train, y_test, "ElasticNet Regression")

TypeError: ElasticNet.__init__() got an unexpected keyword argument 'verbose'

### 5. Decision Tree Regressor

In [12]:
# Decision Tree Regressor
dt_model = DecisionTreeRegressor(max_depth=10, min_samples_split=20, random_state=42)
dt_results = evaluate_model(dt_model, X_train, X_test, y_train, y_test, "Decision Tree Regressor")


Training: Decision Tree Regressor


KeyboardInterrupt: 

KeyboardInterrupt: 

### 6. Random Forest Regressor

In [ ]:
# Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, min_samples_split=10, 
                                  random_state=42, n_jobs=-1, verbose=2)
rf_results = evaluate_model(rf_model, X_train, X_test, y_train, y_test, "Random Forest Regressor")

### 7. Gradient Boosting Regressor

In [ ]:
# Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, 
                                      random_state=42, verbose=1)
gb_results = evaluate_model(gb_model, X_train, X_test, y_train, y_test, "Gradient Boosting Regressor")

### 8. XGBoost Regressor (if available)

In [ ]:
# XGBoost Regressor
if xgboost_available:
    xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, 
                             random_state=42, n_jobs=-1, verbosity=2)
    xgb_results = evaluate_model(xgb_model, X_train, X_test, y_train, y_test, "XGBoost Regressor")
else:
    print("\nSkipping XGBoost (not installed)")
    xgb_results = None

### 9. Support Vector Regressor (SVR)

In [ ]:
# Support Vector Regressor (using subset for faster training)
print("\n⚠️  Note: Training SVR on a subset (50,000 samples) due to computational cost...")

# Use subset for SVR
subset_size = min(50000, len(X_train))
X_train_subset = X_train.iloc[:subset_size]
y_train_subset = y_train.iloc[:subset_size]

svr_model = SVR(kernel='rbf', C=10, gamma='scale', verbose=True)
svr_results = evaluate_model(svr_model, X_train_subset, X_test, y_train_subset, y_test, 
                             "Support Vector Regressor (SVR)")

### 10. K-Nearest Neighbors Regressor

In [ ]:
# K-Nearest Neighbors Regressor
knn_model = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
knn_results = evaluate_model(knn_model, X_train, X_test, y_train, y_test, "K-Nearest Neighbors Regressor")

## Collect All Results

In [ ]:
# Collect all results
all_results = [
    lr_results,
    ridge_results,
    lasso_results,
    elasticnet_results,
    dt_results,
    rf_results,
    gb_results,
    knn_results,
    svr_results
]

if xgb_results:
    all_results.append(xgb_results)

# Create comparison DataFrame
results_df = pd.DataFrame([{
    'Model': r['model_name'],
    'Train RMSE': r['train_rmse'],
    'Test RMSE': r['test_rmse'],
    'Train MAE': r['train_mae'],
    'Test MAE': r['test_mae'],
    'Train R²': r['train_r2'],
    'Test R²': r['test_r2'],
    'Training Time (s)': r['training_time']
} for r in all_results])

# Sort by Test R² (descending)
results_df = results_df.sort_values('Test R²', ascending=False)

print("\n" + "="*100)
print("MODEL COMPARISON SUMMARY")
print("="*100)
print(results_df.to_string(index=False))

# Find best model
best_model_name = results_df.iloc[0]['Model']
best_test_r2 = results_df.iloc[0]['Test R²']
print(f"\n🏆 Best Model: {best_model_name} (Test R² = {best_test_r2:.4f})")

## Visualize Model Performance

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Test R² Comparison
axes[0, 0].barh(results_df['Model'], results_df['Test R²'], color='skyblue', alpha=0.8)
axes[0, 0].set_xlabel('Test R²', fontsize=12)
axes[0, 0].set_title('Model Comparison - Test R² Score', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3, axis='x')
axes[0, 0].invert_yaxis()

# 2. Test RMSE Comparison
axes[0, 1].barh(results_df['Model'], results_df['Test RMSE'], color='coral', alpha=0.8)
axes[0, 1].set_xlabel('Test RMSE', fontsize=12)
axes[0, 1].set_title('Model Comparison - Test RMSE', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3, axis='x')
axes[0, 1].invert_yaxis()

# 3. Test MAE Comparison
axes[1, 0].barh(results_df['Model'], results_df['Test MAE'], color='lightgreen', alpha=0.8)
axes[1, 0].set_xlabel('Test MAE', fontsize=12)
axes[1, 0].set_title('Model Comparison - Test MAE', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3, axis='x')
axes[1, 0].invert_yaxis()

# 4. Training Time Comparison
axes[1, 1].barh(results_df['Model'], results_df['Training Time (s)'], color='gold', alpha=0.8)
axes[1, 1].set_xlabel('Training Time (seconds)', fontsize=12)
axes[1, 1].set_title('Model Comparison - Training Time', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3, axis='x')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Comparison chart saved as 'model_comparison.png'")

## Save Models and Results

In [ ]:
# Save all trained models
print("Saving trained models...\n")

models_to_save = {
    'linear_regression': lr_results,
    'ridge_regression': ridge_results,
    'lasso_regression': lasso_results,
    'elasticnet_regression': elasticnet_results,
    'decision_tree': dt_results,
    'random_forest': rf_results,
    'gradient_boosting': gb_results,
    'knn': knn_results,
    'svr': svr_results
}

if xgb_results:
    models_to_save['xgboost'] = xgb_results

for model_name, results in models_to_save.items():
    with open(f'model_{model_name}.pkl', 'wb') as f:
        pickle.dump(results['model'], f)
    print(f"✓ Saved: model_{model_name}.pkl")

# Save results DataFrame
results_df.to_csv('model_results.csv', index=False)
print(f"\n✓ Saved: model_results.csv")

# Save all results including predictions
with open('all_model_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)
print(f"✓ Saved: all_model_results.pkl")

print("\n" + "="*80)
print("✅ MODEL TRAINING COMPLETE!")
print("="*80)
print(f"\n📁 Generated Files:")
print(f"   • {len(models_to_save)} model files (*.pkl)")
print(f"   • model_results.csv - Summary metrics")
print(f"   • all_model_results.pkl - Complete results with predictions")
print(f"   • model_comparison.png - Visualization")
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Test R²: {best_test_r2:.4f}")
print(f"\n💡 Note: This model predicts song release year from audio features!")